# [How to Do Deep Learning on Graphs with Graph Convolutional Networks](https://github.com/TobiasSkovgaardJepsen/posts/blob/master/HowToDoDeepLearningOnGraphsWithGraphConvolutionalNetworks/Part2_SemiSupervisedLearningWithSpectralGraphConvolutions/notebook.ipynb)
## Part 2: Semi-Supervised Learning with Spectral Graph Convolutions
This notebook accompanies my Medium article with the above title for readers to try out and explore graph convolutional networks for themselves. You can find the article [here](https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-62acf5b143d0). To run the notebook, install the packages specified in the accompanying ```requirements.txt``` file.

#  Load Karate Club

In [1]:
from gcnclass import SpectralRule, LogisticRegressor, load_karate_club

In [2]:
from networkx import to_numpy_matrix, degree_centrality, betweenness_centrality, shortest_path_length
import mxnet.ndarray as nd

zkc = load_karate_club()

A = to_numpy_matrix(zkc.network)
A = nd.array(A)

X_train = zkc.X_train.flatten()
y_train = zkc.y_train
X_test = zkc.X_test.flatten()
y_test = zkc.y_test

# Layer Implementations

In [3]:
from mxnet.gluon import HybridBlock
from mxnet.gluon.nn import Activation
import mxnet.ndarray as nd

# class SpectralRule(HybridBlock):
#     def __init__(self, A, in_units, out_units, activation='relu', **kwargs):
#         super().__init__(**kwargs)
#         I = nd.eye(*A.shape)
#         A_hat = A.copy() + I

#         D = nd.sum(A_hat, axis=0)
#         D_inv = D**-0.5
#         D_inv = nd.diag(D_inv)

#         A_hat = D_inv * A_hat * D_inv
        
#         self.in_units, self.out_units = in_units, out_units
        
#         with self.name_scope():
#             self.A_hat = self.params.get_constant('A_hat', A_hat)
#             self.W = self.params.get(
#                 'W', shape=(self.in_units, self.out_units)
#             )
#             if activation == 'identity':
#                 self.activation = lambda X: X
#             else:
#                 self.activation = Activation(activation)

#     def hybrid_forward(self, F, X, A_hat, W):
#         aggregate = F.dot(A_hat, X)
#         propagate = self.activation(
#             F.dot(aggregate, W))
#         return propagate

In [4]:
class LogisticRegressor(HybridBlock):
    def __init__(self, in_units, **kwargs):
        super().__init__(**kwargs)
        with self.name_scope():
            self.w = self.params.get(
                'w', shape=(1, in_units)
            )

            self.b = self.params.get(
                'b', shape=(1, 1)
            )

    def hybrid_forward(self, F, X, w, b):
        # Change shape of b to comply with MXnet addition API
        b = F.broadcast_axis(b, axis=(0,1), size=(34, 1))
        y = F.dot(X, w, transpose_b=True) + b

        return F.sigmoid(y)

# Models

In [18]:
from mxnet.gluon.nn import HybridSequential, Activation
from mxnet.ndarray import array
from mxnet.initializer import One, Uniform, Xavier
from mxnet.gluon.loss import SigmoidBinaryCrossEntropyLoss

def build_features(A, X):
    hidden_layer_specs = [(4, 'tanh'), (2, 'tanh'),] # Format: (units in layer, activation function)
    in_units = in_units=X.shape[1]
  
    features = HybridSequential()
    with features.name_scope():
        for i, (layer_size, activation_func) in enumerate(hidden_layer_specs):
            layer = SpectralRule(
                A, in_units=in_units, out_units=layer_size, 
                activation=activation_func)
            features.add(layer)

            in_units = layer_size
    return features, in_units

def build_model(A, X):
    model = HybridSequential()
    hidden_layer_specs = [(4, 'tanh'), (2, 'tanh')]
    in_units = in_units=X.shape[1]

    with model.name_scope():
        features, out_units = build_features(A, X)
        model.add(features)

        classifier = LogisticRegressor(out_units)
        model.add(classifier)

    model.hybridize()
    model.initialize(Uniform(1))

    return model, features

## Model 1: Identity Matrix as Features

In [6]:
X_1 = I = nd.eye(*A.shape)
model_1, features_1 = build_model(A, X_1)

In [7]:
model_1(X_1)


[[0.67846113]
 [0.678365  ]
 [0.6784515 ]
 [0.67865455]
 [0.6789896 ]
 [0.6784498 ]
 [0.67840236]
 [0.6778774 ]
 [0.67806447]
 [0.6787555 ]
 [0.6798172 ]
 [0.6771738 ]
 [0.6783973 ]
 [0.67728937]
 [0.6778644 ]
 [0.67681897]
 [0.67838305]
 [0.67841774]
 [0.67886305]
 [0.6782841 ]
 [0.6777961 ]
 [0.6784021 ]
 [0.6793397 ]
 [0.67845035]
 [0.67771035]
 [0.6787817 ]
 [0.67817426]
 [0.6773858 ]
 [0.6777696 ]
 [0.678165  ]
 [0.6781931 ]
 [0.6786611 ]
 [0.6782598 ]
 [0.67848647]]
<NDArray 34x1 @cpu(0)>

## Model 2: Distance to Administrator and Instructor as Additional Features

In [8]:
X_2 = nd.zeros((A.shape[0], 2))
node_distance_instructor = shortest_path_length(zkc.network, target=33)
node_distance_administrator = shortest_path_length(zkc.network, target=0)

for node in zkc.network.nodes():
    X_2[node][0] = node_distance_administrator[node]
    X_2[node][1] = node_distance_instructor[node]

In [9]:
X_2 = nd.concat(X_1, X_2)
model_2, features_2 = build_model(A, X_2)
model_2(X_2)


[[0.41898128]
 [0.42066953]
 [0.4202371 ]
 [0.42225876]
 [0.42629537]
 [0.42489824]
 [0.42482445]
 [0.42436835]
 [0.42339927]
 [0.42722285]
 [0.43601376]
 [0.42917323]
 [0.42348605]
 [0.43088672]
 [0.42715982]
 [0.43249366]
 [0.42259842]
 [0.42457488]
 [0.4318612 ]
 [0.42471617]
 [0.42861357]
 [0.41973785]
 [0.43193817]
 [0.4189893 ]
 [0.43009412]
 [0.43060946]
 [0.4311095 ]
 [0.4307683 ]
 [0.43079725]
 [0.42400852]
 [0.4274449 ]
 [0.42517978]
 [0.42701358]
 [0.43231156]]
<NDArray 34x1 @cpu(0)>

# Train and Test Models

In [10]:
%time
from mxnet import autograd
from mxnet.gluon import Trainer
from mxnet.ndarray import sum as ndsum
import numpy as np

def train(model, features, X, X_train, y_train, epochs):
    cross_entropy = SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)
    trainer = Trainer(model.collect_params(), 'sgd', {'learning_rate': 0.001, 'momentum': 1})

    feature_representations = [features(X).asnumpy()]

    for e in range(1, epochs + 1):
        cum_loss = 0
        cum_preds = []

        for i, x in enumerate(X_train):
            y = array(y_train)[i]
            with autograd.record():
                preds = model(X)[x]
                loss = cross_entropy(preds, y)
            loss.backward()
            trainer.step(1)

            cum_loss += loss[0].asscalar()
            cum_preds += [preds[0].asscalar()]

        feature_representations.append(features(X).asnumpy())
            
        if (e % (epochs//10)) == 0:
            print(f"Epoch {e}/{epochs} -- Loss: {cum_loss: .4f}")
            print(cum_preds)
    return feature_representations

def predict(model, X, nodes):
    preds = model(X)[nodes].asnumpy().flatten()
    return  np.where(preds >= 0.5, 1, 0)

CPU times: user 2 µs, sys: 83 µs, total: 85 µs
Wall time: 5.96 µs


## Performance of Model 1

In [11]:
features_1

HybridSequential(
  (0): SpectralRule(
    (activation): Activation(sigmoid)
  )
  (1): SpectralRule(
    (activation): Activation(sigmoid)
  )
)

In [12]:
X_train


[[ 0.]
 [33.]]
<NDArray 2x1 @cpu(0)>

In [13]:
X_1


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
<NDArray 34x34 @cpu(0)>

In [14]:
from sklearn.metrics import classification_report

feature_representations_1 = train(model_1, features_1, X_1, X_train, y_train, epochs=1000)
y_pred_1 = predict(model_1, X_1, X_test)


Epoch 100/1000 -- Loss:  1.4704
[0.3539598, 0.35067835]
Epoch 200/1000 -- Loss:  1.3379
[0.57393706, 0.5428192]
Epoch 300/1000 -- Loss:  1.1935
[0.6062509, 0.49992096]
Epoch 400/1000 -- Loss:  0.8140
[0.56156254, 0.21099909]
Epoch 500/1000 -- Loss:  0.3472
[0.90632635, 0.2203265]
Epoch 600/1000 -- Loss:  0.1100
[0.95205736, 0.059061453]
Epoch 700/1000 -- Loss:  0.0931
[0.91495174, 0.0041596894]
Epoch 800/1000 -- Loss:  0.0382
[0.9647902, 0.002304521]
Epoch 900/1000 -- Loss:  0.0114
[0.99313897, 0.004508686]
Epoch 1000/1000 -- Loss:  0.0121
[0.998447, 0.010515097]


In [15]:
print(classification_report(y_test.asnumpy(), y_pred_1))

              precision    recall  f1-score   support

         0.0       0.54      0.94      0.68        16
         1.0       0.75      0.19      0.30        16

    accuracy                           0.56        32
   macro avg       0.64      0.56      0.49        32
weighted avg       0.64      0.56      0.49        32



## Performance of Model 2

In [17]:
feature_representations_2= train(model_2, features_2, X_2, X_train, y_train, epochs=250)
y_pred_2 = predict(model_2, X_2, X_test)
print(classification_report(y_test.asnumpy(), y_pred_2))

Epoch 25/250 -- Loss:  1.1695
[0.5862159, 0.47027713]
Epoch 50/250 -- Loss:  1.1351
[0.53844905, 0.4031388]
Epoch 75/250 -- Loss:  1.0982
[0.51091605, 0.34729818]
Epoch 100/250 -- Loss:  1.0190
[0.54106814, 0.33290556]
Epoch 125/250 -- Loss:  0.9039
[0.6200222, 0.3468151]
Epoch 150/250 -- Loss:  0.7781
[0.7011682, 0.34495774]
Epoch 175/250 -- Loss:  0.6298
[0.75172514, 0.29135114]
Epoch 200/250 -- Loss:  0.4786
[0.777421, 0.20294072]
Epoch 225/250 -- Loss:  0.3592
[0.80039114, 0.12764798]
Epoch 250/250 -- Loss:  0.2678
[0.8348302, 0.083568335]
              precision    recall  f1-score   support

         0.0       0.58      0.94      0.71        16
         1.0       0.83      0.31      0.45        16

    accuracy                           0.62        32
   macro avg       0.71      0.62      0.58        32
weighted avg       0.71      0.62      0.58        32

